In [1]:
# Import libraries and custom modules
import numpy as np
import pandas as pd

from src.Data_Retriever import DataRetriever as DatRet

from src.Optimization.Markowitz_PT import MarkowitzPT as MPT

from src.Optimization.Environment import PortfolioEnvironment as PorEnv
from src.Optimization.RLModelCompilation import RL_Model as RLM

from src.Analysis.Menchero_OGA import MencheroOGA as MOGA
from src.Analysis.IndPortResults import GenerateResult as GR
from src.Analysis.OverviewResults import ResultConveyor as RC

import time
"""------------------------------------------------"""
# Start timer
start_time = time.time()
"""------------------------------------------------"""
# Define variables for securitiy pool
n_sectors = 6
n_stocks_per_sector = 3

# Define variables for benchmark
trading_n = 800
history_usage = int(100)

# For RL algorithm
history_usage_RL = 50
rolling_reward_window = 50
"""------------------------------------------------"""
# Defining security pool
ticker_df =  pd.DataFrame({
    "Petroleum": ["EQNR.OL", "SUBC.OL", "BWO.OL"],
    "Food": ["ORK.OL", "MOWI.OL", "LSG.OL"],
    "Materials": ["NHY.OL", "YAR.OL", "RECSI.OL"],
    "Technologies": ["TEL.OL", "NOD.OL", "ATEA.OL"],
    "Financial": ["STB.OL", "DNB.OL", "AKER.OL"],
    "Shipping":["SNI.OL", "BELCO.OL", "ODF.OL"]
})
"""------------------------------------------------"""
# Defining ESG scores for respective securities
esg_scores = np.array([
    36.6, 17.9, 18, 
    18, 23.2, 29.2, 
    15.7, 25.4, 25.6, 
    19.8, 13.8, 18.1, 
    17.3, 14, 12.3, 
    21.2, 26.8, 24.9
])

In [2]:
paths = ["Return_esg_False", "Sharpe_esg_False",
         "Sortino_esg_False","Sterling_esg_False",
         "Calmar_esg_False"]

analysis_list = []
for i in range(len(paths)):
    att_anal = GR(
        paths[i],
        n_sectors, 
        n_stocks_per_sector,
        trading_n,
        esg_scores, 
        ticker_df.columns
    )
    att_anal.friple_frequency_analysis()
    analysis_list.append(att_anal)

----Analysis completed succesfully----
----Analysis completed succesfully----
----Analysis completed succesfully----
----Analysis completed succesfully----
----Analysis completed succesfully----


In [3]:
alfa = analysis_list[0].exper_analysis["sector_allocation"]
bravo = analysis_list[0].exper_analysis["sector_selection"]

In [4]:
alfap = [[alfa[i][j] for i in range(800)] for j in range(6)]
bravop = [[bravo[i][j] for i in range(800)] for j in range(6)]

In [5]:
def create_keywords(input):
    mean = np.round([np.mean(input[i])*10000 for i in range(6)],2)
    std = np.round([np.std(input[i])*10000 for i in range(6)],2)
    cum = np.round([(np.cumprod(np.array(input[i])+1)[-1]-1)*100 for i in range(6)],2)
    return[mean,std,cum]

In [6]:

total = []

for item in range(5):
    alfa = analysis_list[item].exper_analysis["sector_allocation"]
    bravo = analysis_list[item].exper_analysis["sector_selection"]
    alfap = [[alfa[i][j] for i in range(800)] for j in range(6)]
    bravop = [[bravo[i][j] for i in range(800)] for j in range(6)]
    total.append([create_keywords(alfap), create_keywords(bravop)])

In [7]:
this_df = pd.DataFrame(total)
this_df.to_csv("attribution.csv",index=False)